In [812]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ISLP.models import ModelSpec as MS
import statsmodels.api as sm
import keras
from keras import layers
from keras import ops
from sklearn.model_selection import train_test_split
from ISLP import confusion_table

In [813]:
import os
PathToRepo = os.path.normpath(os.getcwd() + os.sep + os.pardir)
Df = pd.read_csv(PathToRepo + '\\Data\\Faellesdata_cleaned.csv')

x = Df.drop('IsMigratorInt', axis=1) 
for i in range(0, x.shape[1]): #Standardizing the data
    x.iloc[:,i] = (x.iloc[:, i] - x.iloc[:, i].mean())/x.iloc[:, i].std()

x_train, x_test, y_train, y_test = train_test_split(x, Df['IsMigratorInt'], test_size=0.2, random_state=42)

In [814]:
x_train = MS(x_train, intercept = False).fit_transform(x_train)

In [815]:
modelnn = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[x_train.shape[1]]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

C:\Users\lorentsen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [816]:
modelnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
modelnn.fit(x_train, y_train, epochs=100)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8118 - loss: 0.4474
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8482 - loss: 0.3265
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8547 - loss: 0.3087
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8608 - loss: 0.2967
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8616 - loss: 0.2885
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8659 - loss: 0.2713
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8699 - loss: 0.2791
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8784 - loss: 0.2662
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8792 - loss: 0.2666
Epoch 10/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8825 - loss: 0.2535
Epoch 11/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8750 - loss: 0.2667
Epoch 12/100
250/250 ━━━━━━━━━━━━━━━━━━━━

In [817]:
modelnn.evaluate(MS(x_test, intercept = False).fit_transform(x_test), y_test)


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8608 - loss: 0.3322


[0.3518635630607605, 0.8495000004768372]

In [818]:
from sklearn.metrics import accuracy_score

In [819]:
confusion_bag = confusion_table(modelnn.predict(MS(x_test, intercept = False).fit_transform(x_test)) > 0.5, y_test)
confusion_bag.to_latex(PathToRepo + '\\Tables\\Confusion1.tex', index_names = True, caption="Confusion table for NN1", label="tab:NN1", position="h!")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [820]:

Models = []
modelconfig = modelnn.get_config()

for i in range(1, len(modelconfig['layers'])):
    neurons = modelconfig['layers'][i]['config']['units']
    activation = modelconfig['layers'][i]['config']['activation']
    layer_type = modelconfig['layers'][i]['class_name']
    Models.append({'Model': 'Neural Network 1', 
                   'Optimizer': modelnn.optimizer.get_config()['name'], 
                   'Loss': modelnn.loss, 
                   'Accuracy': accuracy_score(y_test, modelnn.predict(MS(x_test, intercept = False).fit_transform(x_test)) > 0.5), 
                   'Layer #': i, 'Layer Type': layer_type, 
                   'Neurons': neurons, 
                   'Activation': activation,
                   'epochs': 100})

ModelsDf = pd.DataFrame(Models)


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [821]:
modelsoftmax = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[x_train.shape[1]]),
    layers.Dense(64, activation='relu'),
    layers.Dense(2, activation='softmax')
])

C:\Users\lorentsen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [822]:
modelsoftmax.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
modelsoftmax.fit(x_train, keras.utils.to_categorical(y_train), epochs=100)


Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8244 - loss: 0.4100
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8499 - loss: 0.3199
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8589 - loss: 0.3026
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8599 - loss: 0.2899
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8641 - loss: 0.2869
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8751 - loss: 0.2679
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8781 - loss: 0.2660
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8690 - loss: 0.2758
Epoch 9/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8729 - loss: 0.2635
Epoch 10/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8700 - loss: 0.2626
Epoch 11/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8763 - loss: 0.2579
Epoch 12/100
250/250 ━━━━━━━━━━━━━━━━━━━━

In [823]:
confusion_bag = confusion_table(np.argmax(modelsoftmax.predict(MS(x_test, intercept = False).fit_transform(x_test)), axis = 1)    , y_test)
confusion_bag.to_latex(PathToRepo + '\\Tables\\Confusion2.tex', index_names = True, caption="Confusion table for NN2", label="tab:NN2", position="h!")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


In [824]:
Models = []
modelnn = modelsoftmax

modelconfig = modelnn.get_config()
for i in range(1, len(modelconfig['layers'])):
    neurons = modelconfig['layers'][i]['config']['units']
    activation = modelconfig['layers'][i]['config']['activation']
    layer_type = modelconfig['layers'][i]['class_name']
    Models.append({'Model': 'Neural Network 2'
                   , 'Optimizer': modelnn.optimizer.get_config()['name']
                   , 'Loss': modelnn.loss
                    , 'Accuracy': accuracy_score(y_test, np.argmax(modelsoftmax.predict(MS(x_test, intercept = False).fit_transform(x_test)), axis=1))
                   , 'Layer #': i, 'Layer Type': layer_type, 'Neurons': neurons
                   , 'Activation': activation,
                   'epochs': 100})
NewModelDf = pd.DataFrame(Models)
ModelsDf = ModelsDf.merge(NewModelDf, how='outer', on=['Model', 'Layer #', 'Layer Type', 'Neurons', 'Activation', 'Optimizer', 'Loss', 'Accuracy', 'epochs'])


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


In [825]:
modelnn = keras.Sequential([
    layers.Dense(16, activation='relu', input_shape=[x_train.shape[1]]),
    layers.Dense(1, activation='sigmoid')
])
modelnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
modelnn.fit(x_train, y_train, epochs=100, verbose = 0)
modelnn.evaluate(MS(x_test, intercept = False).fit_transform(x_test), y_test)
modelconfig = modelnn.get_config()



C:\Users\lorentsen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8715 - loss: 0.2645  


In [826]:
confusion_bag = confusion_table(modelnn.predict(MS(x_test, intercept = False).fit_transform(x_test)) > 0.5, y_test)
confusion_bag.to_latex(PathToRepo + '\\Tables\\Confusion3.tex', index_names = True, caption="Confusion table for NN3", label="tab:NN3", position="h!")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [827]:
Models = []
modelconfig = modelnn.get_config()
for i in range(1, len(modelconfig['layers'])):
    neurons = modelconfig['layers'][i]['config']['units']
    activation = modelconfig['layers'][i]['config']['activation']
    layer_type = modelconfig['layers'][i]['class_name']
    Models.append({'Model': 'Neural Network 3', 
                   'Optimizer': modelnn.optimizer.get_config()['name'], 
                   'Loss': modelnn.loss, 
                   'Accuracy': accuracy_score(y_test, modelnn.predict(MS(x_test, intercept = False).fit_transform(x_test)) > 0.5), 
                   'Layer #': i, 'Layer Type': layer_type, 
                   'Neurons': neurons, 
                   'Activation': activation,
                   'epochs': 100})
NewModelDf = pd.DataFrame(Models)
ModelsDf = ModelsDf.merge(NewModelDf, how='outer', on=['Model', 'Layer #', 'Layer Type', 'Neurons', 'Activation', 'Optimizer', 'Loss', 'Accuracy', 'epochs'])


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 857us/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  


In [828]:
modelnn = keras.Sequential([
    layers.Dense(16, activation='relu', input_shape=[x_train.shape[1]]),
    layers.Dropout(0.5),
    layers.Dense(16, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

C:\Users\lorentsen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [829]:
modelnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
modelnn.fit(x_train, y_train, epochs=100, verbose = 0)
modelnn.evaluate(MS(x_test, intercept = False).fit_transform(x_test), y_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8363 - loss: 0.2946 


[0.2923503518104553, 0.8399999737739563]

In [830]:
confusion_bag = confusion_table(modelnn.predict(MS(x_test, intercept = False).fit_transform(x_test)) > 0.5, y_test)
confusion_bag.to_latex(PathToRepo + '\\Tables\\Confusion4.tex', index_names = True, caption="Confusion table for NN4", label="tab:NN4", position="h!")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step  


In [831]:
Models = []
modelconfig = modelnn.get_config()
modelconfig['layers'][2]
for i in range(1, len(modelconfig['layers'])):
    layer_type = modelconfig['layers'][i]['class_name']
    if layer_type == 'Dropout':
        rate = modelconfig['layers'][i]['config']['rate']
        Models.append({'Model': 'Neural Network 4', 
                       'Optimizer': modelnn.optimizer.get_config()['name'], 
                       'Loss': modelnn.loss, 
                       'Accuracy': accuracy_score(y_test, modelnn.predict(MS(x_test, intercept = False).fit_transform(x_test)) > 0.5), 
                       'Layer #': i, 'Layer Type': layer_type, 
                       'Rate': rate,
                        'epochs': 100})
        continue
    neurons = modelconfig['layers'][i]['config']['units']
    activation = modelconfig['layers'][i]['config']['activation']
    Models.append({'Model': 'Neural Network 4', 
                   'Optimizer': modelnn.optimizer.get_config()['name'], 
                   'Loss': modelnn.loss, 
                   'Accuracy': accuracy_score(y_test, modelnn.predict(MS(x_test, intercept = False).fit_transform(x_test)) > 0.5), 
                   'Layer #': i, 'Layer Type': layer_type, 
                   'Neurons': neurons, 
                   'Activation': activation,
                   'epochs': 100})
NewModelDf = pd.DataFrame(Models)
ModelsDf = ModelsDf.merge(NewModelDf, how='outer', on=['Model', 'Layer #', 'Layer Type', 'Neurons', 'Activation', 'Optimizer', 'Loss', 'Accuracy', 'epochs'])


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


Note That the full data file is not included in the repository, as it is too large and that the code below takes over 30 mins to run. The code above just loads the model and calculates the accuracy score.

In [832]:
# from astropy.io import ascii
# FullData = ascii.read(PathToRepo + '\\Data\\m12i_res7100_mhdcv.disk.ecsv', guess = False)
# FullData = FullData.to_pandas()

In [833]:
# FullData['IsMigrator'] = abs(FullData['Rcyl'] - FullData['Rcyl_form']) > 1.5
# FullData['IsMigratorInt'] = FullData['IsMigrator'].astype(int)
# FullData = FullData.drop(['rsph', 'x', 'y',  'vx', 'vy', 
#        'rsph_form', 'x_form', 'y_form', 'z_form', 'vx_form', 'vy_form',
#        'vz_form', 'Rcyl_form', 'phi_form', 'vRcyl_form', 'vphi_form', "IsMigrator"], axis = 1)

In [834]:
# Df = FullData
# x = Df.drop('IsMigratorInt', axis=1) 
# for i in range(0, x.shape[1]): #Standardizing the data
#     x.iloc[:,i] = (x.iloc[:, i] - x.iloc[:, i].mean())/x.iloc[:, i].std()

# x_train_full, x_test_full, y_train_full, y_test_full = train_test_split(x, Df['IsMigratorInt'], test_size=0.2, random_state=42)

In [835]:
# modelnn = keras.Sequential([
#     layers.Dense(16, activation='relu', input_shape=[x_train_full.shape[1]]),
#     layers.Dense(1, activation='sigmoid')
# ])
# modelnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# modelnn.fit(x_train_full, y_train_full, epochs=10)

In [836]:
Models = []
modelnn = keras.models.load_model('modelnn.keras')

modelconfig = modelnn.get_config()
for i in range(1, len(modelconfig['layers'])):
    neurons = modelconfig['layers'][i]['config']['units']
    activation = modelconfig['layers'][i]['config']['activation']
    layer_type = modelconfig['layers'][i]['class_name']
    Models.append({'Model': 'Neural Network 5', 
                   'Optimizer': modelnn.optimizer.get_config()['name'], 
                   'Loss': modelnn.loss, 
                   'Accuracy': accuracy_score(y_test, modelnn.predict(MS(x_test, intercept = False).fit_transform(x_test)) > 0.5), 
                   'Layer #': i, 'Layer Type': layer_type, 
                   'Neurons': neurons, 
                   'Activation': activation,
                   'epochs': 10})
NewModelDf = pd.DataFrame(Models)
ModelsDf = ModelsDf.merge(NewModelDf, how='outer', on=['Model', 'Layer #', 'Layer Type', 'Neurons', 'Activation', 'Optimizer', 'Loss', 'Accuracy', 'epochs'])
confusion_bag = confusion_table(modelnn.predict(MS(x_test, intercept = False).fit_transform(x_test)) > 0.5, y_test)
confusion_bag.to_latex(PathToRepo + '\\Tables\\Confusion5.tex', index_names = True, caption="Confusion table for NN5", label="tab:NN5", position="h!")

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 

C:\Users\lorentsen\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\saving\saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 6 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  


In [837]:
asd = ModelsDf

In [838]:
ModelsDf
ModelsDf.index = ModelsDf['Model']
ModelsDf = ModelsDf.drop('Model', axis = 1)

In [839]:
ModelsDf.index = ModelsDf.index.rename('Model')

In [840]:
ModelsDf.index = pd.MultiIndex.from_tuples([(ModelsDf.index[i],i) for i in range(0, ModelsDf.shape[0])])

In [841]:
ModelsDf['Loss'].replace("_", r"\\textunderscore ", regex=True, inplace=True)

C:\Users\lorentsen\AppData\Local\Temp\ipykernel_51116\2037868839.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  ModelsDf['Loss'].replace("_", r"\\textunderscore ", regex=True, inplace=True)


In [842]:
ModelsDf.to_latex(PathToRepo + '\\Tables\\Models.tex', index_names = True, na_rep='-', multirow = True, float_format="%.3f", caption="Differnet neural networks", label="tab:RawData", position="h!")
ModelsDf

Optimizer                                     Loss  \
Neural Network 1 0       adam       binary\textunderscore crossentropy   
                 1       adam       binary\textunderscore crossentropy   
                 2       adam       binary\textunderscore crossentropy   
Neural Network 2 3       adam  categorical\textunderscore crossentropy   
                 4       adam  categorical\textunderscore crossentropy   
                 5       adam  categorical\textunderscore crossentropy   
Neural Network 3 6       adam       binary\textunderscore crossentropy   
                 7       adam       binary\textunderscore crossentropy   
Neural Network 4 8       adam       binary\textunderscore crossentropy   
                 9       adam       binary\textunderscore crossentropy   
                 10      adam       binary\textunderscore crossentropy   
                 11      adam       binary\textunderscore crossentropy   
                 12      adam       binary\textunderscore crossentropy   
Neural Network 5 13   rmsprop       binary\textunderscore crossentropy   
                 14   rmsprop       binary\textunderscore crossentropy   

                     Accuracy  Layer # Layer Type  Neurons Activation  epochs  \
Neural Network 1 0     0.8495        1      Dense     64.0       relu     100   
                 1     0.8495        2      Dense     64.0       relu     100   
                 2     0.8495        3      Dense      1.0    sigmoid     100   
Neural Network 2 3     0.8550        1      Dense     64.0       relu     100   
                 4     0.8550        2      Dense     64.0       relu     100   
                 5     0.8550        3      Dense      2.0    softmax     100   
Neural Network 3 6     0.8660        1      Dense     16.0       relu     100   
                 7     0.8660        2      Dense      1.0    sigmoid     100   
Neural Network 4 8     0.8400        1      Dense     16.0       relu     100   
                 9     0.8400        2    Dropout      NaN        NaN     100   
                 10    0.8400        3      Dense     16.0       relu     100   
                 11    0.8400        4    Dropout      NaN        NaN     100   
                 12    0.8400        5      Dense      1.0    sigmoid     100   
Neural Network 5 13    0.8800        1      Dense     16.0       relu      10   
                 14    0.8800        2      Dense      1.0    sigmoid      10   

                     Rate  
Neural Network 1 0    NaN  
                 1    NaN  
                 2    NaN  
Neural Network 2 3    NaN  
                 4    NaN  
                 5    NaN  
Neural Network 3 6    NaN  
                 7    NaN  
Neural Network 4 8    NaN  
                 9    0.5  
                 10   NaN  
                 11   0.5  
                 12   NaN  
Neural Network 5 13   NaN  
                 14   NaN